# Lab 4-2: Load Data

#### 학습목표

미니배치 경사하강법(Minibatch Gradient descent)를 배우고 Dataset & DataLoader 사용법에 대해 알아본다.

#### 핵심키워드

- 다항 선형 회귀(Multivariable Linear regression)
- 미니배치 경사하강법(Minibatch Gradient descent)
- Dataset, DataLoader

복잡한 머신러닝 모델을 학습하려면 엄청난 양의 데이터가
필요하다!  

대부분 데이터셋은 적어도 수십만 개의 데이터를 제공한다.   


엄청난 양의 데이터를 한번에 학습시킬 수 없다!
- 너무 느리다.
- 하드웨어적으로 불가능하다.  

그렇다면 일부분의 데이터로만 학습하면 어떨까?

![image](https://user-images.githubusercontent.com/84179578/152271912-66a36b2f-3701-49a4-816a-646c44be012a.png)
![image](https://user-images.githubusercontent.com/84179578/152271934-b0c95f0a-15e5-489c-b228-148fca8143d8.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Dataset and DataLoader

- torch.utils.data.Dataset 상속
- `__len__()`
    - 이 데이터셋의 총 데이터 수
- `__getitem__()`
    - 어떠한 인덱스 idx 를 받았을 때, 그에 상응하는 입출력 데이터 반환
    
    
customdataset 을 구축할때 위의 두가지 magic method 를 필수적으로 지정해줘야한다!  


`__init__` (self, 인수들) : 데이터셋을 처음 선언할 때, 즉 데이터셋 오브젝트가 생길 때 자동으로 불리는 함수이고, 여기에 우리가 몇 가지 인수들을 입력받도록 만들 수 있다 (path, transform 같은 것들).  

`__len__(self)` : 데이터셋의 길이다. 만약 dataset을 선언하고 나서 len(어떤 dataset)을 하면 내부적으로는 이 len 함수가 불리는 것이다. 이 len은 나중에 데이터셋을 선언하고 데이터로더를 사용할 때 또 내부적으로 사용된다. (데이터셋의 len을 알아야 데이터로더가 미니 배치 샘플링을 하면서 지금 다 돌았는지 아닌지를 알 수 있으니까)  

`__getitem__(self, idx)` : 이름에서 알 수 있듯이 데이터셋의 본분인 데이터 하나씩 뽑기이다. idx는 index를 말하는데, 몇 번째 데이터를 뽑을 건지에 대한 변수이다. 이는 데이터로더에서 또 사용될 것이다.

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        
        self.y_data = [[152], [185], [180], [196], [142]]
        
    def __len__(self):
        return len(self.x_data)
        
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y
    
    
dataset = CustomDataset()

dataset 을 만든다음으로 dataloader 를 통해 데이터를 불러올 수 있다  

![image](https://user-images.githubusercontent.com/84179578/152272678-f63e3416-9a71-4c12-9b9d-e1706a9e124f.png)

In [3]:
from torch.utils.data import DataLoader


dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
)

shuffle 은 항상 True 로 하는 것을 추천함

최종적으로 위의 dataset 과 dataloader 를 통해 학습을 할 수 있음

In [4]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력 차원, 출력 차원

    def forward(self, x):
        return self.linear(x)
    
    
# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [5]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))


Epoch    0/20 Batch 1/3 Cost: 4511.914062
Epoch    0/20 Batch 2/3 Cost: 703.825806
Epoch    0/20 Batch 3/3 Cost: 218.549866
Epoch    1/20 Batch 1/3 Cost: 134.377563
Epoch    1/20 Batch 2/3 Cost: 28.320284
Epoch    1/20 Batch 3/3 Cost: 24.841713
Epoch    2/20 Batch 1/3 Cost: 3.070304
Epoch    2/20 Batch 2/3 Cost: 1.786576
Epoch    2/20 Batch 3/3 Cost: 0.008982
Epoch    3/20 Batch 1/3 Cost: 1.254896
Epoch    3/20 Batch 2/3 Cost: 0.506832
Epoch    3/20 Batch 3/3 Cost: 2.196489
Epoch    4/20 Batch 1/3 Cost: 0.849892
Epoch    4/20 Batch 2/3 Cost: 2.248737
Epoch    4/20 Batch 3/3 Cost: 0.088225
Epoch    5/20 Batch 1/3 Cost: 0.336841
Epoch    5/20 Batch 2/3 Cost: 2.605989
Epoch    5/20 Batch 3/3 Cost: 0.286392
Epoch    6/20 Batch 1/3 Cost: 0.924301
Epoch    6/20 Batch 2/3 Cost: 0.911856
Epoch    6/20 Batch 3/3 Cost: 1.871539
Epoch    7/20 Batch 1/3 Cost: 0.449715
Epoch    7/20 Batch 2/3 Cost: 1.413618
Epoch    7/20 Batch 3/3 Cost: 2.158527
Epoch    8/20 Batch 1/3 Cost: 0.851127
Epoch    8/20 